Modificación del programa "Mod4_rew4_umbralsink_enlacePA_a1run.ipynb"
En esta versiÃ³n "b" se elige 1Âº el nodo al que se va a aÃ±adir el nuevo enlace y se va a cambiar la omega

Nuevo enlace Preferential Attachment (m=1)

Actualización: $\Omega = \Omega_0/ (k+1)$;   $\Omega_0=1$

Kuramoto Tradicional ($\alpha=cte=1$)

Condición estabilidad: $r_{SIN} - r_{CON} < \epsilon*r_{sin}$

In [12]:
#import matplotlib
#matplotlib.use('TkAgg')
#from pylab import *
import math
import random as rd
import numpy as np
import networkx as nx
from scipy import stats as st
from numpy import array
from scipy.integrate import ode
from scipy.stats import pareto
import csv

In [13]:
#INTEGRACIÃ“N NUMÃ‰RICA DE LOS OSCILADORES KURAMOTO CUANDO Ã‰STOS ESTÃN EN UNA RED

def sistema():
    
    global g
    
    resultado = []
    
    for i in range(0,len(g.nodes())):
        
        aux = str("g.node["+str(i)+"]['omega']")
        
        for j in range(0,len(g.nodes())):
            
            if g.has_edge(i,j) == True:
                
                aux += str("+alpha*np.sin(P["+str(j)+"]-P["+str(i)+"])")
                
        resultado.append(aux)           
    
    return resultado

def dP_dt2(t,P,alpha,expresion):
    
    resultado = []
    
    for i in range(0,len(g.nodes())):
                
        resultado.append(eval(expresion[i]))           
    
    return resultado

def actualizacion2(g,Tg):
    
    global alpha
    
    expresion = sistema()
    
    #print g.nodes()
    #print expresion

    P0=[]

    for i in g.nodes_iter():

        P0.append(g.node[i]['theta'])

    t0 = 0.0
    t1 = Tg

    solver = ode(dP_dt2)
    solver.set_initial_value(P0, t0)
    solver.set_integrator('vode')
    solver.set_f_params(alpha,expresion)

    while solver.successful() and solver.t < t1:
        solver.integrate(t1, step=True)
   
    for i in g.nodes_iter():
        
        g.node[i]['theta'] = list(array(solver.y))[i] % (2*math.pi)

#INTEGRACIÃ“N NUMÃ‰RICA DE LOS OSCILADORES KURAMOTO CUANDO EL ACOPLAMIENTO DE UN NODO CON SUS VECINOS
#ES PROPORCIONAL A SU GRADO

def sistema_prop_degree():
    
    global g
    
    resultado = []
    
    for i in range(0,len(g.nodes())):
        
        aux = str("g.node["+str(i)+"]['omega']")
        
        for j in range(0,len(g.nodes())):
            
            if g.has_edge(i,j) == True:
                
                aux += str("+alpha*g.degree("+str(i)+")*np.sin(P["+str(j)+"]-P["+str(i)+"])")
                
        resultado.append(aux)           
    sistema_prop_degree
    return resultado

def dP_dt_prop_degree(t,P,alpha,expresion):
    
    resultado = []
    
    for i in range(0,len(g.nodes())):
                
        resultado.append(eval(expresion[i]))           
    
    return resultado

def actualizacion_prop_degree(g,Tg):
    
    global alpha
    
    expresion = sistema_prop_degree()
    
    #print g.nodes()
    #print expresion

    P0=[]

    for i in g.nodes_iter():

        P0.append(g.node[i]['theta'])

    t0 = 0.0
    t1 = Tg

    solver = ode(dP_dt_prop_degree)
    solver.set_initial_value(P0, t0)
    solver.set_integrator('vode')
    solver.set_f_params(alpha,expresion)

    while solver.successful() and solver.t < t1:
        solver.integrate(t1, step=True)
   
    for i in g.nodes_iter():
        
        g.node[i]['theta'] = list(array(solver.y))[i] % (2*math.pi)

In [14]:
def sistema_nodo(nodo):  # genera las ecuaciones (derivada de cada P)
    
    global g
    
    resultado = []
    
    aux = str("g.node["+str(nodo)+"]['omega']")
    
    for i in g.neighbors_iter(nodo):
        
        aux += str("+alpha*sin(g.node["+str(i)+"]['theta']-P[0])") ###  P[]= fase (variable a integrar)
                
    resultado.append(aux)           
    
    return resultado

def dP_dt2_nodo(t,P,alpha,expresion,nodo):
    
    resultado = []
    
    resultado.append(eval(expresion[0]))           
    
    return resultado

def actualizacion2_nodo(g,Tg,nodo):
    
    global alpha
    
    expresion = sistema_nodo(nodo)
    #sistema
    #print g.nodes()
    #print(expresion)

    P0=[]

    P0.append(g.node[nodo]['theta'])

    t0 = 0.0
    t1 = Tg

    solver = ode(dP_dt2_nodo)
    solver.set_initial_value(P0, t0)
    solver.set_integrator('vode')
    solver.set_f_params(alpha,expresion,nodo)

    while solver.successful() and solver.t < t1:
        solver.integrate(t1, step=True)
   
    g.node[nodo]['theta'] = list(array(solver.y))[0]% (2*math.pi) #he añadido "%(2*pi)", JMP-17/06/19

In [15]:
def orden_vecindario(centro): #parÃ¡metro de orden de un nodo y todos sus primeros vecinos
    
    global g
    
    if g.degree(centro) > 0:
    
        vecinos = [centro]

        for i in g.neighbors_iter(centro):

            vecinos.append(i)

        media_sin = 0.0
        media_cos = 0.0

        for i in vecinos:

            media_sin += 1.0 * np.sin(g.node[i]['theta']) / len(vecinos)
            media_cos += 1.0 * np.cos(g.node[i]['theta']) / len(vecinos)

        r_ij = abs(complex(media_cos,media_sin)) #ParÃ¡metro de orden del enlace
        
    else:
        
        r_ij = 1.0
        
    return r_ij

def orden_casi_vecindario(centro,k): #parÃ¡metro de orden de un nodo "centro" y todos sus primeros vecinos, salvo su vecino "k"
    
    global g
    
    if g.degree(centro) > 1.0:
  
        vecinos = [centro]

        for i in g.neighbors_iter(centro):

            if i != k:

                vecinos.append(i)

        media_sin = 0.0
        media_cos = 0.0

        for i in vecinos:

            media_sin += 1.0 * np.sin(g.node[i]['theta']) / len(vecinos)
            media_cos += 1.0 * np.cos(g.node[i]['theta']) / len(vecinos)

        r_ij = abs(complex(media_cos,media_sin)) #ParÃ¡metro de orden del enlace
        
    else:
        
        r_ij = 1.0
        
    return r_ij

In [16]:
def nuevo_enlace(): #Se aÃ±ade al grafo g (con N nodos) un enlace al azar (siempre que sea posible)
    
    global g, N
    
    enl_max = 0.5*N*(N-1) #cantidad mÃ¡xima de enlaces posibles en un grafo g no dirigido, con N nodos
    
    if len(g.edges()) < enl_max:
        
        i = rd.choice(g.nodes())
        
        while g.degree(i) >= N-1: #JMP (2/2/18): Para que coja otro si ya está totalmente enlazado
            
            i = rd.choice(g.nodes())
        
        candidatos = []
        
        for k in nx.non_neighbors(g,i):#Genera un iterador
            
            candidatos.append(k)
        
        j = rd.choice(candidatos)
        
        g.add_edge(i, j)
        
def nuevo_enlace_rnd(i): #Se aÃ±ade al grafo g (con N nodos) un enlace al azar (siempre que sea posible)
    
    global g, N
    
    enl_max = 0.5*N*(N-1) #cantidad mÃ¡xima de enlaces posibles en un grafo g no dirigido, con N nodos
    
    if len(g.edges()) < enl_max:
        
        #i = rd.choice(g.nodes())
        
        while g.degree(i) >= N-1: #JMP (2/2/18): PonÃ­a >=N-1
            
            i = rd.choice(g.nodes())
        
        candidatos = []
        
        for k in nx.non_neighbors(g,i):#Genera un iterador
            
            candidatos.append(k)
        
        j = rd.choice(candidatos)
        
        g.add_edge(i, j)


        
def m_nuevo_enlaces_pref(): #Se añade al grafo g (con N nodos) un enlace (siempre que sea posible) siguiendo el criterio
    # de enlace preferencial.
    
    global g, N, m
    
    enl_max = 0.5*N*(N-1)#cantidad máxima de enlaces posibles en un grafo g no dirigido, con N nodos
    
    if len(g.edges()) < enl_max:
        
        i = rd.choice(g.nodes())
        
        while g.degree(i) >= N-1:
            
            i = rd.choice(g.nodes()) 
            
        #print 'nodo sel: ',i
    
        nds = []

        for j in nx.non_neighbors(g,i):

            nds.append(j)

        fallo = 0.0

        for k in range(m):

            if len(nds) > 0:

                j = pref_select(nds)

                if g.has_edge(i,j) == False and i != j:
                    #print 'enlace: ','(',i,j,')'
                    g.add_edge(i, j)
                    nds.remove(j)

                else:

                    fallo += 1.0

        if fallo > 0:

            print ('Fallo: ',fallo)
            


def m_nuevo_enlaces_pref_i(i): #Se aÃ±ade al grafo g (con N nodos) un enlace (siempre que sea posible) siguiendo el criterio
    # de enlace preferencial.
    #JMP (31/1/18):  SOLO aÃ±ade un enlace y lo hace sobre el nodo "i" que lleva de argumento
    global g, N, m
    
    enl_max = 0.5*N*(N-1)#cantidad mÃ¡xima de enlaces posibles en un grafo g no dirigido, con N nodos
    m_aux = m # JMP(31/1/18). PonÃ­a "m"
    noenlace =-1
    #print('len(g_edges):', len(g.edges()))
    if len(g.edges()) < enl_max:     
        #print('busco vecino, m=',m,'m_aux=',m_aux)
        #Esto es de Sayama, MODELOJUANMA3
        #nds = []
        nds = g.nodes()#      ALF(2/2/18): pARA QUE NO se formen bucles
        #print('nds:',nds)
        for k in range(m_aux):
            #print('k=',k)
            j = pref_select(nds)     
            #print('i', i,' con j:',j)
            if (g.has_edge(i,j) == False) & (j!=i):        
                g.add_edge(i, j)          
                nds.remove(j)   
                #print('enlace con: ',j)
                return j
        
    return noenlace

                
def pref_select(nds):
    global g
    
    r = np.random.uniform(0, sum(g.degree(i) for i in nds))
    x = 0
    for i in nds:
        x += g.degree(i)
        if r <= x:
            return i
            
def actualizar_omega(): #actualiza la velocidad angular (dependiente del grado) tras la adición/corte de enlaces.
    
    global g,cons_omega,gamma
    
    for i in g.nodes_iter():
        
        g.node[i]['omega'] = cons_omega/((g.degree(i)+1)**gamma)   
                    
def actualizar_omega_i(i): #actualiza la velocidad angular (dependiente del grado) tras la adiciÃ³n/corte de enlaces.
    
    global g,cons_omega
    
    #for i in g.nodes_iter():
        
    g.node[i]['omega'] = cons_omega/(g.degree(i)+1)      
    #g.node[i]['omega'] =  g.node[i]['omega'] + np.random.uniform(-delta_omega, delta_omega)/(g.degree(i)+1)
    #for i in g.nodes_iter():       
        #g.node[i]['omega'] = cons_omega/((g.degree(i)+1)**gamma)  Version JUANMA3    


def update_vecindario():
    
    global g, epsilon #umbral   

    rotura = 0
    
    if len(g.edges()) > 0:
            
        lista = []   
        r_con = []

        for i in g.nodes_iter():
            r_con.append(orden_vecindario(i))

        for i in g.nodes_iter():

            grado_nodo = g.degree(i)

            if grado_nodo > 0:

                for j in g.neighbors_iter(i):
                    r_sin = orden_casi_vecindario(j,i)

#                    if (orden_casi_vecindario(j,i) - orden_vecindario(j)) > umbral:
                    if abs(r_sin - r_con[i]) > epsilon* r_sin:
                        # He quitado el ABS para que con el signo negativo no se rompa (14/02/18)
                        #Mod4 (JMP 20/02/18): Lo invierto, "casi_vecindario - vecindario" (vecindario SIN-vecindario CON>umbral => INESTABLE)
                        #print abs(orden_vecindario(i) - orden_casi_vecindario(i,j))
                        e = (i,j)
                        lista.append(e)

        for j in lista:

            if g.has_edge(*j)==True:

                g.remove_edge(*j)
                rotura += 1

        lista_roturas.append(rotura)
    
    if rotura > 0:
        
        return True
    
    else:
        
        False
    
def registro_datos():
    
    global g, lista_cant_nodos,lista_cant_enlaces,lista_roturas,lista_media_sin,lista_media_cos,\
    lista_orden,lista_cant_subgrafos,lista_cantidad_nodos_componente_gigante,lista_diametro_componente_gigante,\
    lista_clustering_medio_componente_gigante,lista_camino_corto_medio_componente_gigante,\
    lista_distancia_enlace_euclidiana_media_componente_gigante,lista_cantidad_enlaces_componente_gigante,\
    componente_gigante
    
    lista_cant_nodos.append(len(g.nodes()))
    lista_cant_enlaces.append(len(g.edges()))
    
    #lista_escisiones.append(escision)
    #lista_roturas.append(rotura) #A: las roturas se registran en la funcion update,\
    #para asÃ­ tener el dato de los instantes de microtiempo
    
    if len(g.nodes()) > 0: #Alf: AÃ±ado un if, por si la red se queda sin nodos
    
        diferencias_sin = 0.0
        diferencias_cos = 0.0

        for i in g.nodes_iter():

            diferencias_sin += np.sin(g.node[i]['theta'])/len(g.nodes())
            diferencias_cos += np.cos(g.node[i]['theta'])/len(g.nodes())

        lista_media_sin.append(diferencias_sin)
        lista_media_cos.append(diferencias_cos)
        lista_orden.append(abs(complex(diferencias_cos,diferencias_sin)))

        ##################################################    #Registro datos sobre los subgrafos y la componente gigante

        subgrafos = sorted(nx.connected_components(g), key = len, reverse=True)

        lista_cant_subgrafos.append(len(subgrafos))

        componente_gigante = g.subgraph(list(subgrafos[0])) #A: Grafo que contiene una copia del componente gigante

        lista_cantidad_nodos_componente_gigante.append(len(componente_gigante.nodes())) #A: Almacena la variable que indica el nombre
        lista_cantidad_enlaces_componente_gigante.append(len(componente_gigante.edges()))
        lista_diametro_componente_gigante.append(nx.diameter(componente_gigante)) #A: Almacena la variable que indica el nombre

        if len(componente_gigante.nodes()) > 1:

            lista_clustering_medio_componente_gigante.append(nx.average_clustering(componente_gigante,weight=None))
            lista_camino_corto_medio_componente_gigante.append(nx.average_shortest_path_length(componente_gigante, weight=None))

        else:

            lista_clustering_medio_componente_gigante.append(float('nan'))
            lista_camino_corto_medio_componente_gigante.append(float('nan'))

        lista_distancia_enlace_euclidiana_media_componente_gigante.append(float('nan'))
    
    
    else: #Alf retoco esto por si la red se queda sin nodos
        
        lista_media_sin.append(float('nan'))
        lista_media_cos.append(float('nan'))
        lista_orden.append(float('nan'))
        lista_cant_subgrafos.append(0.0)
        lista_cantidad_nodos_componente_gigante.append(0.0) #A: Almacena la variable que indica el nombre
        lista_cantidad_enlaces_componente_gigante.append(0.0)
        lista_diametro_componente_gigante.append(float('nan')) #A: Almacena la variable que indica el nombre
        lista_clustering_medio_componente_gigante.append(float('nan'))
        lista_camino_corto_medio_componente_gigante.append(float('nan'))
        lista_distancia_enlace_euclidiana_media_componente_gigante.append(float('nan'))

In [17]:
def initialize():
    global g,N,n0,m,t,delta_theta
    global g, lista_cant_nodos,lista_cant_enlaces,lista_roturas,lista_media_sin,lista_media_cos,\
    lista_orden,lista_cant_subgrafos,lista_cantidad_nodos_componente_gigante,lista_diametro_componente_gigante,\
    lista_clustering_medio_componente_gigante,lista_camino_corto_medio_componente_gigante,\
    lista_distancia_enlace_euclidiana_media_componente_gigante,lista_cantidad_enlaces_componente_gigante
 

    ##################################################  Semilla

    g = nx.Graph() #JJMP(2/2/18) Comienza con una red completa pequeÃ±a
    g  = nx.gnm_random_graph(n0,m0)
    for i in range(0,n0):
        g.node[i]['theta'] = np.random.uniform(-delta_theta, delta_theta)%(2*math.pi) #A: nace con una fase similar a la del
        g.node[i]['omega'] = cons_omega/(g.degree(i)+1)
        #g.node[i]['omega'] = np.random.uniform(-delta_omega, delta_omega)          
    #print('g.node:',g.node)
    
    n=i+1
    while (n<N):        
        g.add_node(n)
        n1=-1
        while n1<0:
            n1=m_nuevo_enlaces_pref_i(n)
        
        #print('n:',n, '; enlaza con n1:',n1, 'tamano:',len(g.nodes()))
        #print('g.node[n]:',g.node[n])
        g.node[n]['theta'] = g.node[n1]['theta']+np.random.uniform(-delta_theta, delta_theta) % (2*math.pi)#A: nace con una fase similar a la del
        g.node[n]['omega'] = cons_omega/(g.degree(i)+1)   
        update_vecindario()

        #g.node[n]['omega'] = g.node[m]['omega']+np.random.uniform(-delta_omega, delta_omega)          
        n=n+1

    t = 0
    tiempo.append(t)
    microtiempo.append(t)
    verificar = update_vecindario()
    while len(g.edges()) > 0 and verificar == True:      
        microtiempo.append(t)
        # JMP (31/1/18): Quito la actualizaciÃ³n de omega de TODA la red, solo el nodo i
        #actualizar_omega()#la velocidad angular es inv. proporcional al grado, asÃ­ que se actualiza tras un cambio de la topologÃ­a.
        verificar = update_vecindario()#se buscan enlaces inestables y, llegado el caso, se cortan

    registro_datos();
#    lista_cant_nodos.append(len(g.nodes()))
#    lista_cant_enlaces.append(len(g.edges()))
#    #lista_escisiones.append(0.0)
#    lista_roturas.append(0.0)
#    lista_cant_subgrafos.append(1.0)
#    lista_media_sin.append(np.sin(g.node[0]['theta'])) #ALF: REVISAR Con una red inicialmente, ese valor no es vÃ¡lido
#    lista_media_cos.append(np.cos(g.node[0]['theta'])) #ALF: REVISAR Con una red inicialmente, ese valor no es vÃ¡lido
#    lista_orden.append(1.0) #ALF: REVISAR Con una red inicialmente, ese valor no es vÃ¡lido
#    lista_cantidad_nodos_componente_gigante.append(1.0) #ALF: REVISAR Con una red inicialmente, ese valor no es vÃ¡lido
#    lista_diametro_componente_gigante.append(float('nan'))
#    tiempo.append(0)
#    microtiempo.append(0)
#    lista_clustering_medio_componente_gigante.append(float('nan'))
#    lista_camino_corto_medio_componente_gigante.append(float('nan'))
#    lista_distancia_enlace_euclidiana_media_componente_gigante.append(float('nan'))
#    lista_cantidad_enlaces_componente_gigante.append(0.0)
    #print len(g.edges())

In [19]:
#############################################################
#############################################################
#PARAMETROS.
#CUIDADO CON EL NOMBRE DE LOS ARCHIVOS DE SALIDA PORQUE LOS MACHACA
#############################################################
#ParÃ¡metros generales
global m

N = 100
#delta_omega = 20 #JMP(6/2/18) Con 0.2 no se rompe ninguno y queda todo atado
delta_theta = math.pi/4
cons_omega = 5.0
gamma=1
############################################################
#Umbrales
epsilon=0.1 #el umbral es epsilon*rparamtro de orden_SIN
#umbral = 0.7 # Diferencia de parametro de orden mayor que umbral se rompe
#m = 2 #cantidad de nuevos enlaces
m = 1 #JMP(31/1/18)  dejo m=1 y cambio la funcion de nuevo enlace para que se le mando el nodo que aÃ±ade nuevo enlace
m0 = 1
n0 = 1
#############################################################
#IntegraciÃ³n numÃ©rica
Tg = 3.0 #duraciÃ³n de la integraciÃ³n numÃ©rica de las ecuaciones diferenciales
alpha = 1.0 # coupling strength
#csvfile = 'salida_listas_Agentes_nn_prueba.txt' #nombre del fichero txt de salida con los datos de las series temporales
#path_graph = 'salida_grafo_Agentes_nn_prueba.gpickle' #nombre del fichero gpickle de salida con el grafo

tiempo = []
microtiempo = [] #Almacena el valor de (macro)tiempo de cada instante de microtiempo
lista_cant_nodos = []
lista_cant_enlaces = []
#lista_escisiones = []
lista_roturas = []
lista_media_sin = []
lista_media_cos = []
lista_orden = []
lista_cant_subgrafos = []
lista_cantidad_nodos_componente_gigante = [] #A: Almacena la variable que indica el nombre
lista_diametro_componente_gigante = [] #A: Almacena la variable que indica el nombre
lista_clustering_medio_componente_gigante = [] #A: Almacena la variable que indica el nombre
lista_camino_corto_medio_componente_gigante = [] #A: Almacena la variable que indica el nombre
lista_distancia_enlace_euclidiana_media_componente_gigante = []  #A: Almacena la variable que indica el nombre
lista_cantidad_enlaces_componente_gigante = []


tmax = 1000

csvfile = 'out_2barrios_usink_eps01_omegadk_PAm1_N100a1om5_t1e3.txt' #nombre del fichero txt de salida con los datos de las series temporales
path_graph = 'outt_2barrios_usink_eps01_omegadk_PAm1_N100a1om5_t1e3.gpickle' #nombre del fichero gpickle de salida con el grafo
path_graph_max = 'out_2barrios_usink_eps01_omegadk_PAm1_N100a1om5_t1e3_max.gpickle' #nombre del fichero gpickle de salida con el grafo

################################
################################
## Comienzo del bucle
################################
################################
#global g,N,m,t #ALF: global es para definir parametros globales dentro de una funciÃ³n. Creo que aquÃ­ no hace falta.
#ALF: Este initialize es el que joroba la lista_roturas_macro. La razÃ³n es que, una vez ejecutado, las listas
#se cargan de valores y luego no se reinicializan
initialize()
g_max = nx.Graph()

if len(g.edges()) > len(g_max.edges()):
        
    g_max = g.copy()
t=1
#registro_datos() #Esta funciÃ³n registra los datos en los instante de macro tiempo
while t < tmax+1:
    
    if t % 100 == 0:     
        print(t)
    
    #for i in range(0,m): #para aÃ±adir m enlaces al azar
        
    #nuevo_enlace()
    i = rd.choice(g.nodes()) ##### 31/1/18 JMP: se elige primero un nodo y sobre ese se aÃ±ade nodoy perturba la omega
    
    #m_nuevo_enlaces_pref_i(i) #para aÃ±adir m enlaces preferencialmente
    m_nuevo_enlaces_pref()
    #nuevo_enlace_rnd(i) #JMP (6/2/18) eNLACE Nuevo Random
    #actualizar_omega_i(i)  #la velocidad angular es inv. proporcional al grado, asÃ­ que se actualiza tras un cambio de la topologÃ­a.

    actualizar_omega() #JMP(26/2/18): Omega = Omega_Cte/(k+1)
    actualizacion2(g,Tg) #integraciÃ³n clÃ¡sica de kuramoto

    #actualizacion_prop_degree(g,Tg) #integraciÃ³n clÃ¡sica de un modelo kuramoto donde el acoplamiento es proporcional al grado
    verificar = update_vecindario() #se buscan enlaces inestables y, llegado el caso, se cortan
    
    #print(len(lista_roturas))

    tiempo.append(t)
    microtiempo.append(t)
    #print(t,microtiempo)
    #print(lista_roturas)
    
    while len(g.edges()) > 0 and verificar == True:
        
        microtiempo.append(t)
        # JMP (31/1/18): Quito la actualizaciÃ³n de omega de TODA la red, solo el nodo i
        #actualizar_omega()#la velocidad angular es inv. proporcional al grado, asÃ­ que se actualiza tras un cambio de la topologÃ­a.
        verificar = update_vecindario()#se buscan enlaces inestables y, llegado el caso, se cortan
        
        
    registro_datos() #Esta funciÃ³n registra los datos en los instante de macro tiempo
    if len(g.edges()) > len(g_max.edges()):
        
        g_max = g.copy()
    

    #print(len(tiempo), len(microtiempo), microtiempo)
    t += 1

    
    if t%100==0:
        
        #print(t)
        #res = [tiempo,microtiempo,lista_cant_nodos,lista_cant_enlaces,lista_roturas,lista_media_sin,lista_media_cos,lista_orden,               lista_cant_subgrafos,lista_cantidad_nodos_componente_gigante,lista_diametro_componente_gigante,lista_clustering_medio_componente_gigante,lista_camino_corto_medio_componente_gigante,               lista_distancia_enlace_euclidiana_media_componente_gigante]
        res=[tiempo,microtiempo,lista_cant_nodos,lista_cant_enlaces,lista_roturas,lista_media_sin,lista_media_cos,\
             lista_orden,lista_cant_subgrafos,lista_cantidad_nodos_componente_gigante,lista_diametro_componente_gigante,\
             lista_clustering_medio_componente_gigante,lista_camino_corto_medio_componente_gigante,\
             lista_distancia_enlace_euclidiana_media_componente_gigante]

        with open(csvfile, "w") as output:
            writer = csv.writer(output, lineterminator='\n')
            writer.writerows(res)
        
        nx.write_gpickle(g, path_graph)
        nx.write_gpickle(g_max, path_graph_max) 
#Salvado final, tras completar la ejecución del programa

#res = [lista_cant_nodos,lista_cant_enlaces,lista_roturas,lista_media_sin,lista_media_cos,lista_orden,       lista_cant_subgrafos,lista_cantidad_nodos_componente_gigante,lista_diametro_componente_gigante,tiempo,       microtiempo,lista_clustering_medio_componente_gigante,lista_camino_corto_medio_componente_gigante,       lista_distancia_enlace_euclidiana_media_componente_gigante]
res=[tiempo,microtiempo,lista_cant_nodos,lista_cant_enlaces,lista_roturas,lista_media_sin,lista_media_cos,\
             lista_orden,lista_cant_subgrafos,lista_cantidad_nodos_componente_gigante,lista_diametro_componente_gigante,\
             lista_clustering_medio_componente_gigante,lista_camino_corto_medio_componente_gigante,\
             lista_distancia_enlace_euclidiana_media_componente_gigante]


with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(res)
    
nx.write_gpickle(g, path_graph) 
nx.write_gpickle(g_max, path_graph_max) 
#initialize()

# In[21]:

#print(nx.neighbors(g,4))       
    
print('t=',t)

100
200
300
400
500
600
700
800
900
1000
('t=', 1001)


In [10]:
for i in range(1):
    print(i)

0
